In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import zoom
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from skimage.measure import block_reduce
import os
import cv2
import glob
from mesh_vox import read_and_reshape_stl, voxelize

In [ ]:
img_path = r"C:\Users\harsh\OneDrive\Desktop\GDrive\tester_folder"
stl_path = r"C:/Users/harsh/OneDrive/Desktop/GDrive/Categorized_Parts"

image = glob.glob(img_path + "/*" + "/*" + "[!.txt]")
stl = glob.glob(stl_path + "/*" + "/STL" + "/*.stl")

In [ ]:
image2 = image[:56] + image[150:206]
stl2 = stl[:56] + stl[150:206]
images2 = np.reshape(image2, (-1,1)).astype(np.object)
stls2 = np.reshape(stl2, (-1,1)).astype(np.object)

In [ ]:
images = np.reshape(image, (-1,1)).astype(np.object)
stls = np.reshape(stl, (-1,1)).astype(np.object)

In [ ]:
def dir_fn(folder):
    folder = folder.decode('utf-8')
    files = tf.gfile.Glob(folder + "\*")
    files = np.reshape(files, (-1,1)).astype(np.object)
    return files

def pre_dir(folder):
    out = tf.py_func(dir_fn, [folder], Tout=tf.string)
    return out

def img_Parse(img):
    img = img[0].decode('utf-8')
    image = cv2.imread(img,0)/255.
    image = cv2.resize(image, (64,64))
    image = np.expand_dims(image,-1)
    image = image.astype(np.float32)
    return image

def preprocess(img):
    out = tf.py_func(img_Parse, [img], Tout=tf.float32)
    return out

def mapping_fn(folder):
    x = tf.map_fn(pre_dir, folder, dtype=np.object)
    image = tf.map_fn(preprocess, x[0], dtype=tf.float32)
    return image

In [ ]:
def stl_parser(stl):
    stl = stl.decode('utf8')
    mesh, _ = read_and_reshape_stl(stl, 16)
    voxels, _ = voxelize(mesh, [16,16,16], False)
    voxels = voxels.astype(np.float32)
    return voxels

def py_fun_stl(stl):
    out = tf.py_func(stl_parser, [stl], Tout=tf.float32)
    return out

def map_fn(stl):
    out = tf.map_fn(py_fun_stl, stl, dtype=tf.float32)
    return out

In [ ]:
class PairGenerator(object):
    
    def __init__(self, images, stl, batch=16):
        self.images = images
        self.stl = stl
        self.batch = batch
        
    def map_fn(self, images, stl):
        img_files = tf.map_fn(pre_dir, images, dtype=np.object)
        image = tf.map_fn(preprocess, img_files[0], dtype=tf.float32)
        voxels = tf.map_fn(py_fun_stl, stl, dtype=tf.float32)
        voxels = tf.squeeze(voxels)
        return (image,voxels)
    
    def datagen(self):
        dataset = tf.data.Dataset.from_tensor_slices((self.images, self.stl))
        dataset = dataset.shuffle(100)
        dataset = dataset.map(self.map_fn).batch(self.batch).repeat()
        dataset = dataset.cache()
        dataset = dataset.prefetch(self.batch)
        iterator = dataset.make_one_shot_iterator()
        return iterator

In [ ]:
class Inputs(object):
    
    def __init__(self, img, stl, batch=32):
        self.img = img
        self.stl = stl
        self.batch = batch
        self.iterator = 0
        self.limit = (len(img)//batch)
        
    def img_parser(self,img):
        image = cv2.imread(img,0)/255.
        image = cv2.resize(image, (64,64))
        image = np.expand_dims(image,-1)
        return image
    
    def _image(self, img_path):
        img_path = glob.glob(img_path + "/*")
        return np.array([self.img_parser(x) for x in img_path])
    
    def stl_parser(self, stl):
        mesh, _ = read_and_reshape_stl(stl, 32)
        voxels, _ = voxelize(mesh, [32,32,32], False)
        return voxels
    
    def pair_generator(self):
        if self.iterator == self.limit:
            self.iterator = 0
        s, e = self.iterator, self.iterator+self.batch
        images = self.img[s:e]
        stls = self.stl[s:e]
        im = np.array([self._image(i) for i in images], dtype=np.float32)
        im = np.swapaxes(im,0,1)
        vx = np.array([self.stl_parser(s) for s in stls], dtype=np.float32)
        self.iterator += 1
        return im, vx

In [ ]:
def Weights(name, shape):
    
    init = tf.random_normal_initializer()
    val = tf.get_variable(name=name, shape=shape, dtype=tf.float32, initializer=init, trainable=True)
    return val    

In [ ]:
class Layer(object):
    
    def __init__(self, prev_layer):
        self._prev_layer = prev_layer
        self._input_shape = prev_layer.output_shape
        self._output = None
        self._out_shape = None
        self._name = None
        
    def set_output(self):
        return
    
    @property
    def output_shape(self):
        if self._out_shape is None:
            self.set_output()
        return self._out_shape
    
    @property
    def output(self):
        if self._output is None:
            self.set_output()
        return self._output

    @property
    def name(self):
        return self._name

In [ ]:
class InputLayer(Layer):
    
    def __init__(self, name, shape, inp=None):
        self._input = inp
        self._out_shape = shape
        self._name = name
        
    @property
    def output_shape(self):
        return self._out_shape
    
    @property
    def output(self):
        return self._input
    
    @property
    def name(self):
        return self._name

In [ ]:
class ConvLayer(Layer):
    
    def __init__(self, name, prev_layer, filter_conf, params=None):
        super().__init__(prev_layer)
        self._filter_shape = [filter_conf[1], filter_conf[1], self._input_shape[-1], filter_conf[0]]
        self._name = name
        
        if params is None:
            self.W = Weights(self._name+"W", self._filter_shape)
            self.b = Weights(self._name+"b", self._filter_shape[-1])
        else:
            self.W = params[0]
            self.b = params[1]
            
        self.params = [self.W, self.b]
        
    def set_output(self):
        self._out_shape = [self._input_shape[0], \
                          self._input_shape[1] - self._filter_shape[0] + 1, \
                           self._input_shape[2] - self._filter_shape[1] + 1, \
                           self._filter_shape[-1]]
        
        conv = tf.nn.conv2d(input= self._prev_layer.output, filter=self.W, strides=[1,1,1,1], padding="VALID")
        self._output = conv + self.b

In [ ]:
class PoolLayer(Layer):
    
    def __init__(self, name, prev_layer, ksize=2, strides=2):
        super().__init__(prev_layer)
        self.ksize = ksize
        self.kernel = [1, ksize, ksize, 1]
        self.strides = [1, strides, strides , 1]
        self._name = name
        
    def set_output(self):
        
        width = self._prev_layer.output_shape[1]
        height = self._prev_layer.output_shape[2]

            
        self._out_shape = [self._prev_layer.output_shape[0], \
                             (width-self.ksize)//2 + 1, (height-self.ksize)//2 + 1, \
                             self._prev_layer.output_shape[3]]
        
        self._output = tf.nn.max_pool(value=self._prev_layer.output, \
                                 ksize=self.kernel, strides=self.strides, \
                                 padding="VALID")

In [ ]:
class FlattenLayer(Layer):
    
    def __init__(self, name, prev_layer):
        super().__init__(prev_layer)
        self._name = name
        
    def set_output(self):
        
        self._out_shape = [self._prev_layer.output_shape[0], \
                          np.prod(self._prev_layer.output_shape[1:])]
        
        self._output = tf.reshape(self._prev_layer.output, self._out_shape)

In [ ]:
class DenseLayer(Layer):
    
    def __init__(self, name, prev_layer, units, params=None):
        super().__init__(prev_layer)
        self._name = name
        self.units = units
        
        if len(self._prev_layer.output_shape) != 2:
            raise ValueError("Add a Flatten Layer before Dense Layer")
            
        if params is None:
            self.W = Weights(self._name+'W', [self._prev_layer.output_shape[1], self.units])
            self.b = Weights(self._name+'b', self.units)
        else:
            self.W = params[0]
            self.b = params[1]
        
        self.params = [self.W, self.b]
        
    def set_output(self):
        
        self._out_shape = [self._prev_layer.output_shape[0], self.units]
        
        self._output = tf.matmul(a=self._prev_layer.output, b=self.W) + self.b

In [ ]:
class ConvTranspose(Layer):
    
    def __init__(self, name, prev_layer, filter_conf, pad="VALID", params=None):
        super().__init__(prev_layer)
        self._name = name
        self.n_kernel = filter_conf[0]
        self.ksize = filter_conf[1]
        self.pad = pad
        self.batch, self.h, self.w, self.d, self.channels = self._prev_layer.output_shape
        
        if params == None:
            self.W = Weights(self._name+"W", [self.ksize, self.ksize, self.ksize, self.n_kernel, \
                                              self._prev_layer.output_shape[-1]])
        else:
            self.W = params
        self.params = self.W
        
        if pad=="VALID":
            padding = self.ksize-1
        else:
            padding = 0
            
        self._out_shape = [self._prev_layer.output_shape[0], \
                          self._prev_layer.output_shape[1] + padding, \
                          self.w + padding, \
                          self.d + padding, \
                          self.n_kernel]
            
    
    def set_output(self):
        
        self._output = tf.nn.conv3d_transpose(self._prev_layer.output, self.W, self._out_shape, strides=(1,1,1,1,1), padding=self.pad)

In [ ]:
def py_func_grad(func, inp, Tout, stateful=True, name=None, grad=None):
    
    rndname = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    
    tf.RegisterGradient(rndname)(grad)
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rndname,
                                "PyFuncStateless": rndname}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def UnPool(prev_out, padding, size):
    b,h,w,d,c = prev_out.shape
    if padding == 0:
        padding = -h
    pad_inp = np.zeros(shape=(b, h*size+2*padding, w*size+2*padding, d*size+2*padding, c), dtype=np.float32)
    pad_inp[:, padding:-padding:size, padding:-padding:size, padding:-padding:size, :] = prev_out
    return pad_inp

def grad_unpool(op, grad):
    x = op.inputs[0]
    p,s = op.inputs[1:]
    z = grad[:, p:-p:s, p:-p:s, p:-p:s, :]
    return x - 0.1*z, p, s

def UnPoolingFunc(prev_out, padding, size, name):
    
    with tf.name_scope(name, 'UnPooling', [prev_out, padding, size]) as name:
        z = py_func_grad(UnPool, [prev_out, padding, size], [tf.float32], name=name, grad=grad_unpool)
        return z[0]

In [ ]:
class UnPooling(Layer):
    
    def __init__(self, name, prev_layer, size, padding=None):
        super().__init__(prev_layer)
        self._name = name
        self.size = size
        self.batch, self.h, self.w, self.d, self.channels = self._prev_layer.output_shape
        
        if padding:
            self.padding = padding
            self._out_shape = [self.batch, self.h*size + 2*padding, self.w*size + 2*padding, \
                              self.d*size + 2*padding, self.channels]
            self.padded_input = tf.Variable(tf.zeros(shape=self._out_shape))
            
        else:
            self.padding = 0
            self._out_shape = [self.batch, self.h*size, self.w*size, self.d*size, self.channels]
            self.padded_input = tf.Variable(tf.zeros(shape=self._out_shape))
        
    def set_output(self):
        
        self._output = UnPoolingFunc(self._prev_layer.output, self.padding, self.size, self._name)

In [ ]:
class ActivationLayer(Layer):
    
    def __init__(self, name, prev_layer, act):
        super().__init__(prev_layer)
        self._name = name
        self.act = act
        self._out_shape = self._prev_layer.output_shape
    
    def set_output(self):
        if self.act == "leakyrelu":
            self._output = tf.nn.leaky_relu(self._prev_layer.output, alpha=0.3)
        elif self.act == "relu":
            self._output = tf.nn.relu(self._prev_layer.output)
        elif self.act == "sigmoid":
            self._output = tf.nn.sigmoid(self._prev_layer.output)
        elif self.act == "softmax":
            self._output = tf.nn.softmax(self._prev_layer.output)
        elif self.act == 'tanh':
            self._output = tf.nn.tanh(self._prev_layer.output)
        else:
            raise ValueError("Use a valid activation function")

In [ ]:
class ExpandLayer(Layer):
    
    def __init__(self, name, prev_layer):
        super().__init__(prev_layer)
        self._name = name
        self._out_shape = self._prev_layer.output_shape + [1]
        
    def set_output(self):
        self._output = tf.expand_dims(self._prev_layer.output, -1)

In [ ]:
class WeightsInitializer(object):
    
    def __init__(self, batch_size=16):
        
        self.batch = batch_size
        self.shape = (12,self.batch,64,64,1)
        self.prev_shape = (self.batch,3,3,3,32)
        self.image = tf.zeros(shape=self.shape)
        
        self.input = InputLayer('input', self.image.shape[1:], self.image[0])
        self.conv1 = ConvLayer('cv1', self.input, (64,5))
        self.pool1 = PoolLayer('pl1', self.conv1, 1, 2)
        self.conv2 = ConvLayer('cv2', self.pool1, (128,5))
        self.pool2 = PoolLayer('pl2', self.conv2, 1, 2)
        self.conv3 = ConvLayer('cv3', self.pool2, (256,3))
        self.pool3 = PoolLayer('pl3', self.conv3, 1, 2)
        self.conv4 = ConvLayer('cv4', self.pool3, (384,3))
        self.pool4 = PoolLayer('pl4', self.conv4, 1, 2)
        #self.conv5 = ConvLayer('cv5', self.pool4, (512,3))
        #self.pool5 = PoolLayer('pl5', self.conv5, 1, 2)
        
        self.flat1 = FlattenLayer('flt1', self.pool4)
        self.dense1 = DenseLayer('ds1', self.flat1, 512)
        self.prev_s = InputLayer('pvs', shape=self.prev_shape, inp=tf.zeros(self.prev_shape))
        self.rnn = RNN('rcn', self.prev_s, self.dense1)
        self.cell = tf.nn.rnn_cell.LSTMCell(864)
        self.deconv1 = ConvTranspose('dcv1', self.rnn, (128,3))
        self.depool1 = UnPooling('upl1', self.deconv1, 2, 1)
        self.deconv2 = ConvTranspose('dcv2', self.depool1, (32,3))
        #self.depool2 = UnPooling('upl2', self.deconv2, 2, padding=1)
        self.deconv3 = ConvTranspose('dcv3', self.deconv2, (16,3))
        #self.depool3 = UnPooling('upl3', self.deconv3, 2, padding=1)
        self.deconv4 = ConvTranspose('dcv4', self.deconv3, (1,1), "SAME")
        #self.deconv5 = ConvTranspose('dcv5', self.deconv4, (1,1), "SAME")

        
def RCNN(image, W, batch):
    
    image = tf.transpose(image, [1,0,2,3,4])
    def Recurrence(prev_s, curr_x):

        #prev_s = InputLayer('ps', prev_s.shape, prev_s)
        input_ = InputLayer('inp', (batch,64,64,1) ,curr_x)

        conv1_ = ConvLayer('conv1', input_, (64,5), params=W.conv1.params)
        aconv1_ = ActivationLayer('aconv1', conv1_, "sigmoid")
        pool1_ = PoolLayer('pool1', aconv1_, 1, 2)

        conv2_ = ConvLayer('conv2', pool1_, (128,5), params=W.conv2.params)
        aconv2_ = ActivationLayer('aconv2', conv2_, "sigmoid")
        pool2_ = PoolLayer('pool2', aconv2_, 1, 2)

        conv3_ = ConvLayer('conv3', pool2_, (256,3), params=W.conv3.params)
        aconv3_ = ActivationLayer('aconv3', conv3_, "sigmoid")
        pool3_ = PoolLayer('pool3', aconv3_, 1, 2)
        
        conv4_ = ConvLayer('conv4', pool3_, (384,3), params=W.conv4.params)
        aconv4_ = ActivationLayer('aconv4', conv4_, "sigmoid")
        pool4_ = PoolLayer('pool4', aconv4_, 1, 2)
        
        #conv5_ = ConvLayer('conv5', pool4_, (512,3), params=W.conv5.params)
        #aconv5_ = ActivationLayer('aconv5', conv5_, "relu")
        #pool5_ = PoolLayer('pool5', aconv5_, 1, 2)

        flat_ = FlattenLayer('flat', pool4_)
        dense_ = DenseLayer('dense', flat_, 512, params=W.dense1.params)
        adense_ = ActivationLayer('adense', dense_, "sigmoid")
        #rnn_ = RNN('rnn', prev_s, adense_, params=W.rnn.params)
        #afc_ = ActivationLayer('afc', rnn_, "sigmoid")

        return adense_.output
    
    #prev_s = W.prev_s
    prev_s = tf.zeros(shape=(batch, 512))
    scn = tf.scan(Recurrence, elems = image, initializer=prev_s)
    rnn_inputs = [tf.squeeze(i,0) for i in tf.split(scn, 12, 0)]
    outputs, final = tf.nn.static_rnn(W.cell, rnn_inputs, dtype=tf.float32)
    final = tf.reshape(final[-1], shape=(batch,3,3,3,32))
    print(final.shape)
    rnn_out = InputLayer('rnnout', (batch,3,3,3,32), final) 
    #return rnn_out
    dconv1_ = ConvTranspose('deconv1', rnn_out, (128,3), params=W.deconv1.params)
    dpool1_ = UnPooling('unpool1', dconv1_, 2, padding=1)
    adpool1_ = ActivationLayer('aunpool1', dpool1_, 'sigmoid')

    dconv2_ = ConvTranspose('deconv2', adpool1_, (32,3), params=W.deconv2.params)
    #dpool2_ = UnPooling('unpool2', dconv2_, 2, padding=1)
    adpool2_ = ActivationLayer('aunpool2', dconv2_, 'sigmoid')

    dconv3_ = ConvTranspose('deconv3', adpool2_, (16,3), params=W.deconv3.params)
    #dpool3_ = UnPooling('unpool3', dconv3_, 1, padding=1)
    adpool3_ = ActivationLayer('aunpool3', dconv3_, 'sigmoid')

    dconv4_ = ConvTranspose('deconv4', adpool3_, (1,1), "SAME", params=W.deconv4.params)
    adconv4_ = ActivationLayer('adeconv4', dconv4_, 'sigmoid')
    #dconv5_ = ConvTranspose('deconv5', adconv4_, (1,1), "SAME", params=W.deconv5.params)
    #adconv5_ = ActivationLayer('adeconv5', dconv5_, 'sigmoid')

    return tf.squeeze(adconv4_.output)

In [ ]:
tf.reset_default_graph()
sess = tf.Session()

In [ ]:
batch = 8
W = WeightsInitializer(batch)
pgen = PairGenerator(images2, stls2, batch)
it = pgen.datagen()
next_element = it.get_next()

In [ ]:
x = tf.placeholder(tf.float32, shape=(batch,12,64,64,1))
y = tf.placeholder(tf.float32, shape=(batch,16,16,16))

In [ ]:
pred = RCNN(x, W, batch)
cost = tf.reduce_mean(tf.losses.mean_squared_error(y, pred))
optimizer = tf.train.AdamOptimizer(0.05).minimize(cost)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
for b in range(100):
    x_, y_ = sess.run(next_element)
    opt = sess.run(optimizer, feed_dict={x: x_, y: y_})
    loss = sess.run(cost, feed_dict={x: x_, y: y_})
    if b%50==0:
        print('Iteration: {} \nLoss: {}\n'.format(b, loss))

In [ ]:
pd = sess.run(pred, feed_dict={x: x_})

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.gca(projection='3d')
ax.voxels(y_[0], edgecolor='k')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.gca(projection='3d')
ax.voxels(pd[7]>0.5, edgecolor='k')
plt.show()